## Problem 1: Geocode shopping centers (5 points)

The overall aim of problems 1-3 is to find out **how many people live within a walking distance (1.5 km) from certain shopping centers in Helsinki**.

In problem 1 aim is to find out the addresses of shopping centers and to retreive numercal coordinates for these addresses. As an output, we will have a Shapefile called `shopping_centers.shp` which contains the geocoded result.

**Preparation:** Find out the addresses for following shopping centers from the internet, and write the addresses into a text file called `shopping_centers.txt`:

 - Itis
 - Forum
 - Iso-omena
 - Sello
 - Jumbo
 - REDI
 - Tripla 
 
 *Hint for "Tripla": This shopping center opened in October 10 2019, and if you are doing this exercise soon after that, the official address might not yet be in online databases. 
 Check for an address nearby the Pasila railway station on OpenStreetMap.org and use that as input.*

`shopping_centers.txt` should have semicolon (`;`) as a separator, and the file should include the following columns:

- ``id`` (integer) containing an unique identifier for each shopping center
- ``name`` (string) of each shopping center
- ``addr`` (string) the address 


See and example of how to format the text file [in the lesson 3 materials](https://automating-gis-processes.github.io/site/master/notebooks/L3/geocoding_in_geopandas.html). Save (and upload) the text file into your exercise repository.

- Read `shopping_centers.txt` that you just created into a pandas DataFrame called ``data``:

In [58]:
import pandas as pd

data = pd.read_csv(r'C:\Users\Maria\Desktop\ISB\GIS Course\Exercise-3-master\Exercise-3-master\shopping_centers.txt', sep =';')


In [59]:
#NON-EDITABLE TEST CELL
# Check your input data
print(data)

     id       name                                             addr
0  1000       Itis             Itäkatu 1-7, 00930 Helsinki, Finland
1  1001      Forum   Mannerheimintie 14–20, 00100 Helsinki, Finland
2  1002  Iso-omena            Piispansilta 11, 02230 Espoo, Finland
3  1003      Sello        Leppävaarankatu 3-9, 02600 Espoo, Finland
4  1004      Jumbo       Vantaanportinkatu 3, 01510 Vantaa, Finland
5  1005       REDI    Hermannin rantatie 5, 00580 Helsinki, Finland
6  1006     Tripla          Ratapihantie 6, 00520 Helsinki, Finland


- Geocode the addresses using the Nominatim geocoding service. Store the output in a variable called `geo`:

In [60]:
from geopandas.tools import geocode

geo = geocode(data[' addr'],provider='nominatim', user_agent='autogis_ms', timeout=10) 

In [61]:
#NON-EDITABLE TEST CELL
# Check the geocoded output
print(geo)

                    geometry  \
0  POINT (25.08294 60.21170)   
1  POINT (24.93828 60.16878)   
2  POINT (24.73907 60.16091)   
3  POINT (24.81187 60.21823)   
4  POINT (24.96282 60.29245)   
5  POINT (24.98072 60.18731)   
6  POINT (24.93435 60.19857)   

                                             address  
0  Kauppakeskus Itis, Itäkatu, Itäkeskus, Vartiok...  
1  Salaattiasema, Mannerheimintie, Keskusta, Kluu...  
2  11, Piispansilta, Nuottaniemi, Piispansilta, E...  
3  3-9, Leppävaarankatu, Etelä-Leppävaara, Leppäv...  
4  Stockmann, Vantaanportinkatu, Pakkala, Aviapol...  
5  XXL Redi, Hermannin rantatie, Kalasatama, Sörn...  
6  6, Ratapihantie, Itä-Pasila, Pasila, Helsinki,...  


In [62]:
#NON-EDITABLE TEST CELL
# Check the data type (should be a GeoDataFrame!)
print(type(geo))

<class 'geopandas.geodataframe.GeoDataFrame'>


Check that the coordinate reference system of the geocoded result is correctly defined, and **reproject the layer into ETRS GK-25** (EPSG:3879):

In [63]:
geo = geo.to_crs(epsg=3879)

In [64]:
#NON-EDITABLE TEST CELL
# Check layer crs
print(geo.crs)

epsg:3879


- Make a table join between the geocoded addresses (``geo``) and the original addresses (``data``) in order to link the numerical coordinates and  the `id` and `name` of each shopping center. 
- Store the output in a variable called ``geodata`` 


In [65]:
geodata = geo.join(data)

In [66]:
#NON-EDITABLE TEST CELL
# Check the join output
print(geodata.head())

                           geometry  \
0  POINT (25504598.602 6677662.109)   
1  POINT (25496573.542 6672878.360)   
2  POINT (25485510.634 6672029.255)   
3  POINT (25489571.409 6678401.654)   
4  POINT (25497943.932 6686656.982)   

                                             address    id       name  \
0  Kauppakeskus Itis, Itäkatu, Itäkeskus, Vartiok...  1000       Itis   
1  Salaattiasema, Mannerheimintie, Keskusta, Kluu...  1001      Forum   
2  11, Piispansilta, Nuottaniemi, Piispansilta, E...  1002  Iso-omena   
3  3-9, Leppävaarankatu, Etelä-Leppävaara, Leppäv...  1003      Sello   
4  Stockmann, Vantaanportinkatu, Pakkala, Aviapol...  1004      Jumbo   

                                              addr  
0             Itäkatu 1-7, 00930 Helsinki, Finland  
1   Mannerheimintie 14–20, 00100 Helsinki, Finland  
2            Piispansilta 11, 02230 Espoo, Finland  
3        Leppävaarankatu 3-9, 02600 Espoo, Finland  
4       Vantaanportinkatu 3, 01510 Vantaa, Finland  


- Save the output as a Shapefile called `shopping_centers.shp` 

In [20]:
out_fp = r'C:\Users\Maria\Desktop\ISB\GIS Course\Exercise-3-master\Exercise-3-master\shopping_centers.shp'
geodata.to_file(out_fp)

In [67]:
#NON-EDITABLE TEST CELL
# Print info about output file
print("Geocoded output is stored in this file:", out_fp)

Geocoded output is stored in this file: C:\Users\Maria\Desktop\ISB\GIS Course\Exercise-3-master\Exercise-3-master\shopping_centers.shp


## Problem 2: Create buffers around shopping centers (5 points)

Let's continue with our case study and calculate a 1.5 km buffer around the geocoded points. 


- Start by creating a new column called `buffer` to ``geodata`` GeoDataFrame:

In [68]:
import geopandas as gpd
buffer = []

- Calculate a 1.5 km buffer for each geocoded point. Store the buffer geometry in the new `buffer` column.

Here, you can use the [GeoDataFrame buffer() method](http://geopandas.org/geometric_manipulations.html#GeoSeries.buffer), which uses Shapely's [buffer](http://toblerity.org/shapely/manual.html#object.buffer) in the bacground. You only need to use the `distance` -parameter, don't worry about the other parameters.

In [69]:
buffer = geodata.buffer(1500)
geodata['buffer'] = buffer


In [70]:
#NON-EDITABLE TEST CELL
print(geodata.head())

                           geometry  \
0  POINT (25504598.602 6677662.109)   
1  POINT (25496573.542 6672878.360)   
2  POINT (25485510.634 6672029.255)   
3  POINT (25489571.409 6678401.654)   
4  POINT (25497943.932 6686656.982)   

                                             address    id       name  \
0  Kauppakeskus Itis, Itäkatu, Itäkeskus, Vartiok...  1000       Itis   
1  Salaattiasema, Mannerheimintie, Keskusta, Kluu...  1001      Forum   
2  11, Piispansilta, Nuottaniemi, Piispansilta, E...  1002  Iso-omena   
3  3-9, Leppävaarankatu, Etelä-Leppävaara, Leppäv...  1003      Sello   
4  Stockmann, Vantaanportinkatu, Pakkala, Aviapol...  1004      Jumbo   

                                              addr  \
0             Itäkatu 1-7, 00930 Helsinki, Finland   
1   Mannerheimintie 14–20, 00100 Helsinki, Finland   
2            Piispansilta 11, 02230 Espoo, Finland   
3        Leppävaarankatu 3-9, 02600 Espoo, Finland   
4       Vantaanportinkatu 3, 01510 Vantaa, Finland   

 

In [71]:
#NON-EDITABLE TEST CELL
# Check the data type of the first value in the buffer-column
print(type(geodata.at[0,'buffer']))

<class 'shapely.geometry.polygon.Polygon'>


In [72]:
#NON-EDITABLE TEST CELL
# Check the areas of your buffers in km^2
print(round(gpd.GeoSeries(geodata["buffer"]).area / 1000000))

0    7.0
1    7.0
2    7.0
3    7.0
4    7.0
5    7.0
6    7.0
dtype: float64


- Replace the values in `geometry` column with the values of `buffer` column:

In [73]:
geodata['geometry'] = buffer

In [74]:
#NON-EDITABLE TEST CELL
print(geodata.head())

                                            geometry  \
0  POLYGON ((25506098.602 6677662.109, 25506091.3...   
1  POLYGON ((25498073.542 6672878.360, 25498066.3...   
2  POLYGON ((25487010.634 6672029.255, 25487003.4...   
3  POLYGON ((25491071.409 6678401.654, 25491064.1...   
4  POLYGON ((25499443.932 6686656.982, 25499436.7...   

                                             address    id       name  \
0  Kauppakeskus Itis, Itäkatu, Itäkeskus, Vartiok...  1000       Itis   
1  Salaattiasema, Mannerheimintie, Keskusta, Kluu...  1001      Forum   
2  11, Piispansilta, Nuottaniemi, Piispansilta, E...  1002  Iso-omena   
3  3-9, Leppävaarankatu, Etelä-Leppävaara, Leppäv...  1003      Sello   
4  Stockmann, Vantaanportinkatu, Pakkala, Aviapol...  1004      Jumbo   

                                              addr  \
0             Itäkatu 1-7, 00930 Helsinki, Finland   
1   Mannerheimintie 14–20, 00100 Helsinki, Finland   
2            Piispansilta 11, 02230 Espoo, Finland   
3       

Optional: at this point, you can drop out unnecessary columns from the geodataframe. In the next problem, 
we will only need these columns: `'id', 'name', 'geometry'`

In [75]:
geodata = geodata.drop(['buffer','address',' addr'], axis = 1)
geodata

,geometry,id,name
0,"POLYGON ((25506098.602 6677662.109, 25506091.3...",1000,Itis
1,"POLYGON ((25498073.542 6672878.360, 25498066.3...",1001,Forum
2,"POLYGON ((25487010.634 6672029.255, 25487003.4...",1002,Iso-omena
3,"POLYGON ((25491071.409 6678401.654, 25491064.1...",1003,Sello
4,"POLYGON ((25499443.932 6686656.982, 25499436.7...",1004,Jumbo
5,"POLYGON ((25500430.407 6674941.797, 25500423.1...",1005,REDI
6,"POLYGON ((25497858.800 6676198.280, 25497851.5...",1006,Tripla


## Problem 3: How many people live near shopping centers? (5 points)

Last step in our analysis is to make a spatial join between our buffer layer and population data in order to find out **how many people live near each shopping center**. We will use the same data as we did during [lesson 3](https://automating-gis-processes.github.io/site/notebooks/L3/spatial-join.html#Spatial-join); **a Population Grid** that is available via the HSY wfs. 

Alternatively, you can also download the data from the [Helsinki Region Infoshare (HRI)](https://www.hsy.fi/fi/asiantuntijalle/avoindata/Sivut/AvoinData.aspx?dataID=7) as a shapefile (using wget).

The coordinate reference system of the population grid is **ETRS GK-25 (EPSG:3879)**.


**Steps:**

- Read the population grid into a geodataframe

- Select only the useful columns from the population grid: ``'asukkaita'`` (=population count per grid square) and ``'geometry'`` 

- Make a spatial join between your buffered point layer and population grid layer. Join the information now from buffer layer **into the population grid layer**

- Group the joined layer by shopping center index

- Calculate the sum of population living within 1.5 km for each shopping center.

**Finally:**

- Print out the population living within 1.5 km from each shopping center:

     - Itis
     - Forum
     - Iso-omena
     - Sello
     - Jumbo
     - REDI
     - Tripla
     
**Final print out should contain both the shopping center name and population count**, for example: `25858 people live within 1.5 km from Iso-Omena`.

*Feel free to divide your solution into several codeblocks! Remember to comment your code  :)*

In [76]:
# Read population grid data for 2018 into a variable `pop`. 
# Remember to check the crs info! 

from pyproj import CRS
import requests
import geojson

# Specify the url for web feature service
url = 'https://kartta.hsy.fi/geoserver/wfs'

# Specify parameters (read data in json format).
# Available feature types in this particular data source: http://geo.stat.fi/geoserver/vaestoruutu/wfs?service=wfs&version=2.0.0&request=describeFeatureType
params = dict(service='WFS',
              version='2.0.0',
              request='GetFeature',
              typeName='asuminen_ja_maankaytto:Vaestotietoruudukko_2018',
              outputFormat='json')

# Fetch data from WFS using requests
r = requests.get(url, params=params)

# Create GeoDataFrame from geojson
pop = gpd.GeoDataFrame.from_features(geojson.loads(r.content))

In [82]:
pop = pop[['geometry','asukkaita']]

In [84]:
#NON-EDITABLE TEST CELL
# Check your input data
print("Number of rows:", len(pop))
print(pop.head(3))

Number of rows: 3167
                                            geometry  asukkaita
0  MULTIPOLYGON Z (((25476499.999 6674248.999 0.0...        108
1  MULTIPOLYGON Z (((25476749.997 6674498.998 0.0...        273
2  MULTIPOLYGON Z (((25476999.994 6675749.004 0.0...        239


In [87]:
pop.crs = CRS.from_epsg(3879).to_wkt()

In [89]:
pop.crs == geodata.crs

True

In [93]:
# Create a spatial join between grid layer and buffer layer. 

join = gpd.sjoin(geodata, pop)

In [99]:
grouped = join.groupby('name')

In [105]:
grouped.sum()

,id,index_right,asukkaita
name,,,
REDI,46230,86852,39190
Sello,68204,62963,27053
Tripla,69414,113904,44800
Forum,90090,149206,78018
Iso-omena,93186,48533,33295
Itis,85000,237480,27951
Jumbo,27108,47839,10317


In [107]:
# Report how many people live within 1.5 km distance from each shopping center

for index,row in grouped.sum().iterrows():
    print(row['asukkaita'],' people live within 1.5 km from ',index)

39190  people live within 1.5 km from   REDI
27053  people live within 1.5 km from   Sello
44800  people live within 1.5 km from   Tripla
78018  people live within 1.5 km from  Forum
33295  people live within 1.5 km from  Iso-omena
27951  people live within 1.5 km from  Itis
10317  people live within 1.5 km from  Jumbo


**Reflections:**
    
- How challenging did you find problems 1-3 (on scale to 1-5), and why?
- What was easy?
- What was difficult?

YOUR ANSWER HERE

Well done! Now you can continue to [problem 4](Exercise-3-Problem-4.ipynb)